In [1]:
%cd /hpc2hdd/home/yliu364/Qwen-VL
!pip install -r requirements.txt

/hpc2hdd/home/yliu364/Qwen-VL
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install deepspeed peft einops_exts

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
torch.manual_seed(1234)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True).eval()

# query = tokenizer.from_list_format([
#     {'image': image_path}, # Either a local path or an url
#     {'text': 'What is it?'},
# ])



The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Loading checkpoint shards: 100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


<ref> Search icon</ref><box>(312,100),(449,183)</box>


In [31]:
image_path = '/hpc2hdd/home/yliu364/3.jpg'
response, history = model.chat(tokenizer, query=f'<img>{image_path}</img>Please give me the coordinates of "Search icon"', history=None)
print(response)

image = tokenizer.draw_bbox_on_latest_picture(response, history)
if image:
  image.save('output_2.jpg')
else:
  print("no box")

<ref> Search icon</ref><box>(851,496),(933,538)</box>


In [7]:
from peft import AutoPeftModelForCausalLM
from transformers.generation import GenerationConfig
import torch
model_rico = AutoPeftModelForCausalLM.from_pretrained(
    "/hpc2hdd/home/yliu364/Qwen-VL/output_qwen_chat/checkpoint-400", # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Loading checkpoint shards: 100%|██████████| 10/10 [00:17<00:00,  1.72s/it]
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 151860. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [26]:
from transformers import AutoTokenizer
tokenizer_rico = AutoTokenizer.from_pretrained("/hpc2hdd/home/yliu364/Qwen-VL/output_qwen_chat/checkpoint-400", trust_remote_code=True)
image_path = '/hpc2hdd/home/yliu364/3.jpg'
response, history = model_rico.chat(tokenizer, query=f'<img>{image_path}</img>Please give me the coordinates of "Search icon"', history=None)
print(response)

<ref> Search icon</ref><box>(852,495),(941,538)</box>


In [27]:
image = tokenizer.draw_bbox_on_latest_picture(response, history)
if image:
  image.save('output_rico_2.jpg')
else:
  print("no box")